In [1]:
# import here
import pandas as pd
from sqlalchemy import create_engine
from references_dict import Team_Dictionary
from calculate_skillpoints import SkillPoints

# connect to databases
kaggle_engine = create_engine('mysql+pymysql://root:@localhost:3306/kaggle')
kaggle_conn = kaggle_engine.connect()
main_engine = create_engine('mysql+pymysql://root:@localhost:3306/main_stats')
main_conn = main_engine.connect()

In [2]:
# home stats
home_rush_sql = "select game_id, posteam as home_abbrev, sum(yards_gained) as home_rush_yds, sum(rush_attempt) as home_rush_att,sum(rush_touchdown) as home_rush_tds \
  from nfl_pbp \
  where posteam = home_team \
  and play_type = 'run' \
group by game_id"

home_short_pass_sql = "select game_id,sum(yards_gained) as home_short_pass_yds \
  from nfl_pbp \
  where play_type='pass' \
  and pass_length = 'short' \
  and posteam=home_team \
  group by game_id,posteam"

home_deep_pass_sql = "select game_id,sum(yards_gained) as home_deep_pass_yds \
  from nfl_pbp \
  where play_type='pass' \
  and pass_length = 'deep' \
  and posteam=home_team \
  group by game_id,posteam"

home_pass_total_sql = "select game_id, \
       sum(pass_attempt) as home_pass_att, \
       sum(complete_pass) as home_completions, \
       sum(pass_touchdown) as home_pass_tds, \
       sum(sack) as home_sacked, \
       sum(interception) as home_interceptions \
  from nfl_pbp \
  where play_type = 'pass' \
  and posteam = home_team \
  group by game_id"

home_rush_mets = pd.read_sql_query(home_rush_sql, kaggle_conn, index_col=None)
home_rush_mets.set_index('game_id',inplace=True)
home_short_pass_mets = pd.read_sql_query(home_short_pass_sql, kaggle_conn, index_col=None)
home_short_pass_mets.set_index('game_id',inplace=True)
home_deep_pass_mets = pd.read_sql_query(home_deep_pass_sql, kaggle_conn, index_col=None)
home_deep_pass_mets.set_index('game_id',inplace=True)
home_pass_mets = pd.read_sql_query(home_pass_total_sql, kaggle_conn, index_col=None)
home_pass_mets.set_index('game_id',inplace=True)

home_offense = home_rush_mets.merge(home_short_pass_mets,on='game_id')
home_offense = home_offense.merge(home_deep_pass_mets,on='game_id')
home_offense = home_offense.merge(home_pass_mets,on='game_id')

home_offense.head()

,home_abbrev,home_rush_yds,home_rush_att,home_rush_tds,home_short_pass_yds,home_deep_pass_yds,home_pass_att,home_completions,home_pass_tds,home_sacked,home_interceptions
game_id,,,,,,,,,,,
2009091000,PIT,36.0,23.0,0.0,218.0,145.0,47.0,33.0,1.0,4.0,2.0
2009091300,ATL,72.0,24.0,0.0,207.0,22.0,38.0,22.0,2.0,2.0,0.0
2009091301,BAL,198.0,41.0,2.0,195.0,112.0,44.0,26.0,3.0,1.0,1.0
2009091302,CAR,87.0,29.0,1.0,94.0,39.0,39.0,14.0,0.0,5.0,5.0
2009091303,CIN,86.0,27.0,1.0,247.0,0.0,36.0,21.0,0.0,3.0,2.0


In [3]:
# home stats
away_rush_sql = "select game_id, \
    posteam as away_abbrev, \
    sum(yards_gained) as away_rush_yds, \
    sum(rush_attempt) as away_rush_att, \
    sum(rush_touchdown) as away_rush_tds \
  from nfl_pbp \
  where posteam = away_team \
  and play_type = 'run' \
group by game_id"

away_short_pass_sql = "select game_id,sum(yards_gained) as away_short_pass_yds \
  from nfl_pbp \
  where play_type='pass' \
  and pass_length = 'short' \
  and posteam = away_team \
  group by game_id,posteam"

away_deep_pass_sql = "select game_id,sum(yards_gained) as away_deep_pass_yds \
  from nfl_pbp \
  where play_type='pass' \
  and pass_length = 'deep' \
  and posteam = away_team \
  group by game_id,posteam"

away_pass_total_sql = "select game_id, \
       sum(pass_attempt) as away_pass_att, \
       sum(complete_pass) as away_completions, \
       sum(pass_touchdown) as away_pass_tds, \
       sum(sack) as away_sacked, \
       sum(interception) as away_interceptions \
  from nfl_pbp \
  where play_type = 'pass' \
  and posteam = away_team \
  group by game_id"

away_rush_mets = pd.read_sql_query(away_rush_sql, kaggle_conn, index_col=None)
away_rush_mets.set_index('game_id',inplace=True)
away_short_pass_mets = pd.read_sql_query(away_short_pass_sql, kaggle_conn, index_col=None)
away_short_pass_mets.set_index('game_id',inplace=True)
away_deep_pass_mets = pd.read_sql_query(away_deep_pass_sql, kaggle_conn, index_col=None)
away_deep_pass_mets.set_index('game_id',inplace=True)
away_pass_mets = pd.read_sql_query(away_pass_total_sql, kaggle_conn, index_col=None)
away_pass_mets.set_index('game_id',inplace=True)

away_offense = away_rush_mets.merge(away_short_pass_mets,on='game_id')
away_offense = away_offense.merge(away_deep_pass_mets,on='game_id')
away_offense = away_offense.merge(away_pass_mets,on='game_id')

away_offense.head()

,away_abbrev,away_rush_yds,away_rush_att,away_rush_tds,away_short_pass_yds,away_deep_pass_yds,away_pass_att,away_completions,away_pass_tds,away_sacked,away_interceptions
game_id,,,,,,,,,,,
2009091000,TEN,86.0,25.0,0.0,152.0,92.0,36.0,22.0,1.0,1.0,1.0
2009091300,MIA,96.0,22.0,0.0,139.0,37.0,33.0,21.0,1.0,4.0,1.0
2009091301,KC,29.0,17.0,0.0,127.0,50.0,27.0,16.0,2.0,3.0,0.0
2009091302,PHI,186.0,31.0,1.0,102.0,0.0,31.0,17.0,2.0,2.0,1.0
2009091303,DEN,75.0,20.0,0.0,106.0,137.0,32.0,17.0,1.0,3.0,0.0


In [4]:
all_offense = home_offense.merge(away_offense,on='game_id')

In [5]:
all_offense.head()

,home_abbrev,home_rush_yds,home_rush_att,home_rush_tds,home_short_pass_yds,home_deep_pass_yds,home_pass_att,home_completions,home_pass_tds,home_sacked,...,away_rush_yds,away_rush_att,away_rush_tds,away_short_pass_yds,away_deep_pass_yds,away_pass_att,away_completions,away_pass_tds,away_sacked,away_interceptions
game_id,,,,,,,,,,,,,,,,,,,,,
2009091000,PIT,36.0,23.0,0.0,218.0,145.0,47.0,33.0,1.0,4.0,...,86.0,25.0,0.0,152.0,92.0,36.0,22.0,1.0,1.0,1.0
2009091300,ATL,72.0,24.0,0.0,207.0,22.0,38.0,22.0,2.0,2.0,...,96.0,22.0,0.0,139.0,37.0,33.0,21.0,1.0,4.0,1.0
2009091301,BAL,198.0,41.0,2.0,195.0,112.0,44.0,26.0,3.0,1.0,...,29.0,17.0,0.0,127.0,50.0,27.0,16.0,2.0,3.0,0.0
2009091302,CAR,87.0,29.0,1.0,94.0,39.0,39.0,14.0,0.0,5.0,...,186.0,31.0,1.0,102.0,0.0,31.0,17.0,2.0,2.0,1.0
2009091303,CIN,86.0,27.0,1.0,247.0,0.0,36.0,21.0,0.0,3.0,...,75.0,20.0,0.0,106.0,137.0,32.0,17.0,1.0,3.0,0.0


In [6]:
df = SkillPoints(all_offense).df

In [7]:
df.head()

,home_abbrev,home_rush_yds,home_rush_att,home_rush_tds,home_short_pass_yds,home_deep_pass_yds,home_pass_att,home_completions,home_pass_tds,home_sacked,...,away_deep_pass_yds,away_pass_att,away_completions,away_pass_tds,away_sacked,away_interceptions,rush_skillpoints_home,rush_skillpoints_away,pass_skillpoints_home,pass_skillpoints_away
game_id,,,,,,,,,,,,,,,,,,,,,
2009091000,PIT,36.0,23.0,0.0,218.0,145.0,47.0,33.0,1.0,4.0,...,92.0,36.0,22.0,1.0,1.0,1.0,5.165217,12.040000,57.470213,40.844444
2009091300,ATL,72.0,24.0,0.0,207.0,22.0,38.0,22.0,2.0,2.0,...,37.0,33.0,21.0,1.0,4.0,1.0,10.200000,13.963636,45.636842,30.963636
2009091301,BAL,198.0,41.0,2.0,195.0,112.0,44.0,26.0,3.0,1.0,...,50.0,27.0,16.0,2.0,3.0,0.0,36.629268,4.605882,61.063636,36.181481
2009091302,CAR,87.0,29.0,1.0,94.0,39.0,39.0,14.0,0.0,5.0,...,0.0,31.0,17.0,2.0,2.0,1.0,17.700000,30.600000,3.325641,27.522581
2009091303,CIN,86.0,27.0,1.0,247.0,0.0,36.0,21.0,0.0,3.0,...,137.0,32.0,17.0,1.0,3.0,0.0,17.785185,11.250000,29.950000,36.331250


In [ ]:
df.to_sql('nfl_gamestats', con=main_engine, if_exists='replace',index='game_id')